In [4]:
import time
import traceback
import datetime
import copy
import sys
import pickle
import os
import yaml
import pandas as pd
import numpy as np
import lightgbm as lgb

import ccxt

In [6]:
# 必要な情報を読み込み
def fetch_yaml_config():
    """read yaml config file"""
    config = "config.yml"
    with open(config, "r", encoding="utf-8") as yml:
        config = yaml.safe_load(yml)
    return config

In [12]:
def market_order(exchange, symbol, lot, api_key=None, api_secret=None):
    print("order",lot)
    if exchange == "bitflyer":
        config = {"apiKey": api_key, "secret": api_secret,"options": {"defaultType": "future"}}
        ex = ccxt.binance(config)

        side = "Buy" if lot > 0 else "Sell"
        if side == "Sell":
            ord = ex.create_market_sell_order(symbol, abs(lot))
        else:
            ord = ex.create_market_buy_order(symbol, lot)

        return ord

In [13]:
def test_order(exchange, symbol, api_key, api_secret):
    market_order(exchange, symbol, lot, api_key, api_secret)
    market_order(exchange, symbol, -lot, api_key, api_secret)

In [20]:
if __name__ == "__main__":
    config = fetch_yaml_config()
    # 環境変数を設定する
    api_key = config["API_KEY"]
    api_secret = config["API_SECRET"]
    lot = float(config["LOT"])
    symbol = config["SYMBOL"]
    exchange = config["EXCHANGE"]

    periods_dict = {"1m" :60, "3m":60*3, "5m":60*5, "15m":60*15, "30m":60*30, "1h":60*60, "2h":60*60*2, "4h":60*60*4, "1d":60*60*24}

    period = "1m"
    timespan = periods_dict[period]
    t = time.time()
    pre_time = t//timespan
    #test_order(exchange, symbol, api_key, api_secret)
    # while True:
    #     t = time.time()
    #     now_time = t//timespan
    #     if pre_time != now_time:
    #         pre_time = now_time
    #         run(exchange, symbol, api_key, api_secret)
    #     time.sleep(1)

In [15]:
print(ccxt.exchanges)

['aax', 'ascendex', 'bequant', 'bibox', 'bigone', 'binance', 'binancecoinm', 'binanceus', 'binanceusdm', 'bit2c', 'bitbank', 'bitbay', 'bitbns', 'bitcoincom', 'bitfinex', 'bitfinex2', 'bitflyer', 'bitforex', 'bitget', 'bithumb', 'bitmart', 'bitmex', 'bitopro', 'bitpanda', 'bitrue', 'bitso', 'bitstamp', 'bitstamp1', 'bittrex', 'bitvavo', 'bkex', 'bl3p', 'blockchaincom', 'btcalpha', 'btcbox', 'btcmarkets', 'btctradeua', 'btcturk', 'buda', 'bw', 'bybit', 'bytetrade', 'cdax', 'cex', 'coinbase', 'coinbaseprime', 'coinbasepro', 'coincheck', 'coinex', 'coinfalcon', 'coinmate', 'coinone', 'coinspot', 'crex24', 'cryptocom', 'currencycom', 'delta', 'deribit', 'digifinex', 'eqonex', 'exmo', 'flowbtc', 'fmfwio', 'ftx', 'ftxus', 'gateio', 'gemini', 'hitbtc', 'hitbtc3', 'hollaex', 'huobi', 'huobijp', 'huobipro', 'idex', 'independentreserve', 'indodax', 'itbit', 'kraken', 'kucoin', 'kucoinfutures', 'kuna', 'latoken', 'lbank', 'liquid', 'luno', 'lykke', 'mercado', 'mexc', 'ndax', 'novadax', 'oceanex',